In [19]:
# importing pandas, requests, os and json
import requests
import json
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [20]:
foursquare_key = os.getenv('foursquare_key')

In [25]:
def get_foursquare_data(api_key, bike_stations, radius=1000):
    '''
    Makes requests to the Foursquare API for each bike station in my city_bikes_Montreal.csv file.
    
    Args:
        api_key (str): The Foursquare API key.
        bike_stations (pd.DataFrame): A DataFrame containing bike station data.
        radius (int): The search radius in meters around each bike station. Defaults to 1000.

    Returns:
        list: A list of JSON responses from the Foursquare API for each bike station.
    '''
    
    base_url = "https://api.foursquare.com/v3/places/search"
    headers = {
        "accept": "application/json",
        "Authorization": api_key
                }
    responses = []

    for index, row in bike_stations.iterrows():
        url = f"{base_url}?ll={row['latitude']},{row['longitude']}&radius={radius}"
        response = requests.get(url, headers=headers)
        responses.append(response.json())

    return responses

# Path to CSV file
file_path = '/path/to/your/city_bikes_Montreal.csv'

# Read the CSV into a DataFrame
bike_stations = pd.read_csv('../data/city_bikes_Montreal.csv')

# Foursquare API key
# I am having trouble
foursquare_key = os.getenv('foursquare_key')

# Call the function
foursquare_responses = get_foursquare_data(foursquare_key, bike_stations)


In [26]:
print(foursquare_responses)

[{'results': [{'fsq_id': '4ff33b5fe4b0aeae9cc8fb6d', 'categories': [{'id': 10001, 'name': 'Amusement Park', 'short_name': 'Amusement Park', 'plural_name': 'Amusement Parks', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/themepark_', 'suffix': '.png'}}, {'id': 13018, 'name': 'Pub', 'short_name': 'Pub', 'plural_name': 'Pubs', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'short_name': 'Restaurant', 'plural_name': 'Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 145, 'geocodes': {'main': {'latitude': 45.516495, 'longitude': -73.565819}, 'roof': {'latitude': 45.516495, 'longitude': -73.565819}}, 'link': '/v3/places/4ff33b5fe4b0aeae9cc8fb6d', 'location': {'address': '2041 Saint-Denis Rue', 'country': 'CA', 'cross_street': '', 'formatted_address': '2041 S

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [29]:
def parse_response(foursquare_responses):
    '''
    Parses the Foursquare API responses to extract specific POI details.

    Args:
        foursquare_responses (list): A list of JSON responses from the Foursquare API.

    Returns:
        pd.DataFrame: A DataFrame containing the parsed POI data with specific columns.
    '''

    pois_list = []

    for response in foursquare_responses:
        for item in response.get('results', []):
            poi_info = {
                'name': item.get('name'),
                'category': item.get('categories', [{}])[0].get('name', 'N/A'),
                'address': item.get('location', {}).get('address', 'N/A'),
                'country': item.get('location', {}).get('country', 'N/A'),
                'latitude': item.get('geocodes', {}).get('main', {}).get('latitude'),
                'longitude': item.get('geocodes', {}).get('main', {}).get('longitude')
            }
            pois_list.append(poi_info)

    return pd.DataFrame(pois_list)

# [Rest of the script to load data and call get_foursquare_data]

# Parse the responses
foursquare_parsed_data = parse_response(foursquare_responses)

Put your parsed results into a DataFrame

In [30]:
foursquare_parsed_data

,name,category,address,country,latitude,longitude
0,Randolph Pub Ludique,Amusement Park,2041 Saint-Denis Rue,CA,45.516495,-73.565819
1,Le 4e Mur,Music Venue,2021 Saint-Denis Rue,CA,45.516216,-73.565568
2,Patrick'S Pub Irlandais,Irish Pub,1707 Saint-Denis Rue,CA,45.515508,-73.564043
3,Le Cheval Blanc,Bar,809 Ontario Rue E,CA,45.518170,-73.564467
4,Bibliothèque et Archives nationales du Québec,Library,475 Blvd de Maisonneuve Est,CA,45.515487,-73.562251
...,...,...,...,...,...,...
1588,Nacarat,Cocktail Bar,900 René-Lévesque O,CA,45.500712,-73.568076
1589,La Maison Simons,Clothing Store,977 Sainte-Catherine Rue O,CA,45.501138,-73.572303
1590,Nyks Bistro Pub,Bar,1250 Rue de Bleury,CA,45.506024,-73.566648
1591,Cineplex Inc,Movie Theater,977 Sainte-Catherine Rue O,CA,45.501026,-73.572202


In [32]:
# Download the DataFrame to a CSV file for later use.

foursquare_parsed_data.to_csv('foursquare_data.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating